# Azure OpenAI Assistants

Azure OpenAI Assistants allows you to create AI assistants tailored to your needs through custom instructions and augmented by advanced tools like code interpreter, and custom functions. In this article we'll provide an in-depth walkthrough of getting started with the Assistants API.

In [1]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version=os.getenv("API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

# Create an assistant
assistant = client.beta.assistants.create(
    name="Data Visualization",
    instructions=f"You are a helpful AI assistant who makes interesting visualizations based on data." 
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a visualization you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. If the code is successful display the visualization."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[{"type": "code_interpreter"}],
    model="GPT41106" #You must replace this value with the deployment name of your model.
)

In [2]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_XYKKVtvnyIxW11gLqBNubsW2",
  "created_at": 1711544196,
  "description": null,
  "file_ids": [],
  "instructions": "You are a helpful AI assistant who makes interesting visualizations based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. If the code is successful display the visualization.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "metadata": {},
  "model": "GPT41106",
  "name": "Data Visualization",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ]
}


In [3]:
# Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_uRygNGkvCqEeY0yevCGjhkWa', created_at=1711544208, metadata={}, object='thread')


In [4]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a visualization of a sinewave"
)

In [5]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_O8NYIvds5VBDpcsGfBXw2R08",
      "assistant_id": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Create a visualization of a sinewave"
          },
          "type": "text"
        }
      ],
      "created_at": 1711544225,
      "file_ids": [],
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_uRygNGkvCqEeY0yevCGjhkWa"
    }
  ],
  "object": "list",
  "first_id": "msg_O8NYIvds5VBDpcsGfBXw2R08",
  "last_id": "msg_O8NYIvds5VBDpcsGfBXw2R08",
  "has_more": false
}


We could also pass an instructions parameter here, but this would override the existing instructions that we have already provided for the assistant.

In [21]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [23]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

completed


In [24]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_ulP1SAqgGbItvMcXjURh9ELI",
      "assistant_id": "asst_XYKKVtvnyIxW11gLqBNubsW2",
      "completed_at": null,
      "content": [
        {
          "image_file": {
            "file_id": "assistant-s5B1r68rP7KZEPQLoE8Khvxh"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is the visualization of a sine wave, as requested."
          },
          "type": "text"
        }
      ],
      "created_at": 1711547469,
      "file_ids": [],
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_5lBCHJXUVIKoKOwtXoqMt2gX",
      "status": null,
      "thread_id": "thread_uRygNGkvCqEeY0yevCGjhkWa"
    },
    {
      "id": "msg_GF3VF29XMNpLBgQsrNsDchi9",
      "assistant_id": "asst_XYKKVtvnyIxW11gLqBNubsW2",
      "completed_at": null,
      "content": [
    

In [25]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(data)  

{'data': [{'id': 'msg_ulP1SAqgGbItvMcXjURh9ELI', 'assistant_id': 'asst_XYKKVtvnyIxW11gLqBNubsW2', 'completed_at': None, 'content': [{'image_file': {'file_id': 'assistant-s5B1r68rP7KZEPQLoE8Khvxh'}, 'type': 'image_file'}, {'text': {'annotations': [], 'value': 'Here is the visualization of a sine wave, as requested.'}, 'type': 'text'}], 'created_at': 1711547469, 'file_ids': [], 'incomplete_at': None, 'incomplete_details': None, 'metadata': {}, 'object': 'thread.message', 'role': 'assistant', 'run_id': 'run_5lBCHJXUVIKoKOwtXoqMt2gX', 'status': None, 'thread_id': 'thread_uRygNGkvCqEeY0yevCGjhkWa'}, {'id': 'msg_GF3VF29XMNpLBgQsrNsDchi9', 'assistant_id': 'asst_XYKKVtvnyIxW11gLqBNubsW2', 'completed_at': None, 'content': [{'text': {'annotations': [], 'value': 'It seems like the issue has been resolved. Let me try to run the code again to create a visualization of the sine wave.'}, 'type': 'text'}], 'created_at': 1711547453, 'file_ids': [], 'incomplete_at': None, 'incomplete_details': None, 'me

In [26]:
content = client.files.content(image_file_id)

image= content.write_to_file("sinewave.png")

In [27]:
from PIL import Image

# Display the image in the default image viewer
image = Image.open("sinewave.png")
image.show()

In [28]:
# Add a new user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the code you used to generate the sinewave"
)

In [29]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions  but these will override the default instructions
)


In [31]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
status = run.status
print(status)

completed


* Please wait until you get "completed"

In [32]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_sPz4Rrzge2Fgw7dH8pUPKtFP",
      "assistant_id": "asst_XYKKVtvnyIxW11gLqBNubsW2",
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Certainly! Here is the code I used to generate the sine wave visualization:\n\n```python\nimport numpy as np\nimport matplotlib.pyplot as plt\n\n# Generate a sine wave\nx = np.linspace(0, 2 * np.pi, 100)  # Create an array of x values from 0 to 2*pi\ny = np.sin(x)  # Compute the sine of each x value\n\n# Plot the sine wave\nplt.figure(figsize=(10, 5))  # Set the figure size\nplt.plot(x, y)  # Plot x against y\nplt.title('Sine Wave')  # Give the plot a title\nplt.xlabel('x (radians)')  # Label the x-axis\nplt.ylabel('sin(x)')  # Label the y-axis\nplt.grid(True)  # Show a grid for reference\nplt.show()  # Display the plot\n```\n\nThis code uses `numpy` to generate the x values and compute the y values for the sine function, and `matplotlib.pyp

In [33]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

Certainly! Here is the code I used to generate the sine wave visualization:

```python
import numpy as np
import matplotlib.pyplot as plt

# Generate a sine wave
x = np.linspace(0, 2 * np.pi, 100)  # Create an array of x values from 0 to 2*pi
y = np.sin(x)  # Compute the sine of each x value

# Plot the sine wave
plt.figure(figsize=(10, 5))  # Set the figure size
plt.plot(x, y)  # Plot x against y
plt.title('Sine Wave')  # Give the plot a title
plt.xlabel('x (radians)')  # Label the x-axis
plt.ylabel('sin(x)')  # Label the y-axis
plt.grid(True)  # Show a grid for reference
plt.show()  # Display the plot
```

This code uses `numpy` to generate the x values and compute the y values for the sine function, and `matplotlib.pyplot` to create the plot and display it.


In [34]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I prefer visualizations in darkmode can you change the colors to make a darkmode version of this visualization."
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [43]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

completed


* Please wait until you get "completed"

In [45]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_huV6yUZ5co1iUTHVxk7Op30p",
      "assistant_id": "asst_XYKKVtvnyIxW11gLqBNubsW2",
      "completed_at": null,
      "content": [
        {
          "image_file": {
            "file_id": "assistant-1SIsijROUAEnUujjVTb4QxNp"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is the dark mode version of the sine wave visualization, featuring a dark background and lighter, contrasting colors for the sine wave and text."
          },
          "type": "text"
        }
      ],
      "created_at": 1711563080,
      "file_ids": [],
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_MlgQ2d3wYPOb3rgzEDHO9wHD",
      "status": null,
      "thread_id": "thread_uRygNGkvCqEeY0yevCGjhkWa"
    },
    {
      "id": "msg_BpJYKsMEDNtxoxPyEujzVvxK",
      "assis

In [46]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id'] # index numbers can vary if you have had a different conversation over the course of the thread.

print(image_file_id)

content = client.files.content(image_file_id)
image= content.write_to_file("dark_sine.png")

# Display the image in the default image viewer
image = Image.open("dark_sine.png")
image.show()

assistant-1SIsijROUAEnUujjVTb4QxNp


* You can delete the resource

In [48]:
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_XYKKVtvnyIxW11gLqBNubsW2', deleted=True, object='assistant.deleted')